In [1]:
import pandas as pd
from tqdm import tqdm
from music_plots import load
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

files = os.listdir('data/spectrograms')
tracks_df = load("data/tracks.csv")
genres_df = load("data/genres.csv")

track_ids = []
print(files)
for file in tqdm(files):
    pre, ext = os.path.splitext(file)
    track_ids.append(int(pre))

filtered_tracks = tracks_df.xs('track', level=0, axis=1)['genre_top'].loc[track_ids]
filtered_tracks

genres = {}
tmp_x = 0
#Assign value for the genres
#for i in genres_df[genres_df['parent']==0]['title'].unique():
for i in filtered_tracks.unique():
    genres[i] = tmp_x
    tmp_x = tmp_x + 1

print(genres)
genres_df[genres_df['parent']==0]

#train_ds = filtered_tracks.sample(frac = 0.8)

#tmp_ds = filtered_tracks.drop(train_ds.index)
#validation_ds = tmp_ds.sample(frac = 0.5)
#test_ds = tmp_ds.drop(validation_ds.index)

removed = filtered_tracks.sample(frac = 0.8)
tmp_ds = filtered_tracks.drop(removed.index)
train_ds = tmp_ds.sample(frac=0.8)
tmp_ds = tmp_ds.drop(train_ds.index)
validation_ds = tmp_ds.sample(frac = 0.5)
test_ds = tmp_ds.drop(validation_ds.index)

print(f"Training has {len(train_ds)}, Validation has {len(validation_ds)}, Testing has {len(test_ds)}")

tf_train_data = pd.DataFrame(columns=['Filename', 'label'])
tf_validation_data = pd.DataFrame(columns=['Filename', 'label'])
tf_test_data = pd.DataFrame(columns=['Filename', 'label'])
for i in tqdm(train_ds.index):
    tf_train_data.loc[i]=["data/spectrograms/" + str(i) + ".png", genres[train_ds.loc[i]]]

for i in tqdm(validation_ds.index):
    tf_validation_data.loc[i]=["data/spectrograms/" + str(i) + ".png", genres[validation_ds.loc[i]]]

for i in tqdm(test_ds.index):
    tf_test_data.loc[i]=["data/spectrograms/" + str(i) + ".png", genres[test_ds.loc[i]]]

train_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1 / 255.,           # normalize pixel values between 0-1
    rotation_range=0,
)

validation_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1 / 255.,           # normalize pixel values between 0-1
    rotation_range=0,
)

test_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1 / 255.,           # normalize pixel values between 0-1
    rotation_range=0
)

BATCH_SIZE = 32
print(list(genres.keys()))

traingen = train_generator.flow_from_dataframe(tf_train_data,class_mode='raw',
                                               x_col='Filename',
                                               y_col = 'label',
                                               subset='training',
                                               shuffle=False,
                                               batch_size=BATCH_SIZE,
                                               target_size=(174,484))


validationgen = validation_generator.flow_from_dataframe(tf_validation_data,class_mode='raw',
                                               x_col='Filename',
                                               y_col = 'label',
                                               subset='training',
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                                         target_size=(174,484))

testgen = test_generator.flow_from_dataframe(tf_test_data,class_mode='raw',
                                               x_col='Filename',
                                               y_col = 'label',
                                               subset='training',
                                               shuffle=False,
                                             batch_size=BATCH_SIZE,
                                             target_size=(174,484))

2022-07-01 16:21:01.310303: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 16:21:01.310357: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['112000.png', '95452.png', '134449.png', '127878.png', '51157.png', '145704.png', '110756.png', '15210.png', '107909.png', '52502.png', '126749.png', '58063.png', '46241.png', '129694.png', '120775.png', '57937.png', '114297.png', '146343.png', '133573.png', '145705.png', '144935.png', '107534.png', '114410.png', '53862.png', '145706.png', '74445.png', '53457.png', '73584.png', '147126.png', '43029.png', '707.png', '3912.png', '111938.png', '29526.png', '120194.png', '125723.png', '133793.png', '86678.png', '17605.png', '65265.png', '145002.png', '29740.png', '73921.png', '52639.png', '40938.png', '108493.png', '77952.png', '109962.png', '48452.png', '48367.png', '98551.png', '108301.png', '18033.png', '29741.png', '48440.png', '131658.png', '115815.png', '40244.png', '29528.png', '24365.png', '85428.png', '21167.png', '92554.png', '136323.png', '62446.png', '146716.png', '14569.png', '143995.png', '50753.png', '114064.png', '116709.png', '127281.png', '48494.png', '79975.png', '14661

100%|██████████| 7847/7847 [00:00<00:00, 297725.89it/s]


{'Instrumental': 0, 'Experimental': 1, 'Pop': 2, 'Hip-Hop': 3, 'Electronic': 4, 'Folk': 5, 'Rock': 6, 'International': 7}
Training has 1255, Validation has 157, Testing has 157


100%|██████████| 157/157 [00:00<00:00, 343.74it/s]

['Instrumental', 'Experimental', 'Pop', 'Hip-Hop', 'Electronic', 'Folk', 'Rock', 'International']
Found 1255 validated image filenames.
Found 157 validated image filenames.
Found 157 validated image filenames.


In [2]:
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras
from keras.applications import ResNet50

def extract_features(model, generator):
    features = model.predict(generator)
    features = features.reshape((features.shape[0], -1))
    return features

base_model = ResNet50(weights="imagenet")
print(base_model.summary())
base_model = keras.Model(base_model.input, base_model.get_layer("conv5_block1_2_relu").output)
print(base_model.summary())
y_train = tf_train_data["label"]
X_train = extract_features(base_model, traingen)
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train = std_scaler.transform(X_train)
print(X_train.shape)
y_test = tf_test_data["label"]
X_test = extract_features(base_model, testgen)
X_test = std_scaler.transform(X_test)

2022-06-29 23:13:50.721343: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-29 23:13:50.721391: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-29 23:13:50.721428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wonderland): /proc/driver/nvidia/version does not exist
2022-06-29 23:13:50.722119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [3]:
svm_params = {"C": [0.01, 0.1, 1, 10], "gamma": [0.01, 0.001, 0.0001]}

In [4]:
svm_model = GridSearchCV(SVC(kernel="rbf"), svm_params, scoring="accuracy", n_jobs=-1, cv=2)
svm_model.fit(X_train, y_train)
prediction = svm_model.predict(X_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.38      0.31      0.34        26
           1       0.15      0.11      0.12        19
           2       0.13      0.15      0.14        13
           3       0.52      0.62      0.57        21
           4       0.33      0.47      0.39        19
           5       0.47      0.40      0.43        20
           6       0.48      0.68      0.57        19
           7       0.58      0.35      0.44        20

    accuracy                           0.39       157
   macro avg       0.38      0.39      0.37       157
weighted avg       0.40      0.39      0.39       157



In [5]:
pca_scaler = PCA(0.90)
pca_scaler.fit(X_train)
X_train_PCA = pca_scaler.transform(X_train)
X_test_PCA = pca_scaler.transform(X_test)

print(X_test_PCA.shape)
print(X_train_PCA.shape)
print(X_test.shape)
print(X_train.shape)
svm_model = GridSearchCV(SVC(kernel="rbf"), svm_params, scoring="accuracy", n_jobs=-1, cv=10)
svm_model.fit(X_train_PCA, y_train)
prediction = svm_model.predict(X_test_PCA)
print(classification_report(y_test, prediction))

(157, 177)
(1255, 177)
(157, 49152)
(1255, 49152)
              precision    recall  f1-score   support

           0       0.47      0.31      0.37        26
           1       0.20      0.11      0.14        19
           2       0.15      0.15      0.15        13
           3       0.55      0.76      0.64        21
           4       0.39      0.47      0.43        19
           5       0.50      0.55      0.52        20
           6       0.47      0.74      0.57        19
           7       0.62      0.40      0.48        20

    accuracy                           0.45       157
   macro avg       0.42      0.44      0.41       157
weighted avg       0.43      0.45      0.43       157



In [6]:
tf_test_data["label"]

23156     7
75211     6
131476    0
82930     0
145728    3
         ..
99274     1
121661    0
10384     5
49067     1
26464     5
Name: label, Length: 157, dtype: int64